In [ ]:
import numpy as np
import itertools
from deepwalk import graph
from gensim.models import Word2Vec
import os
import psutil
from tqdm import tqdm
import networkx as nx
import pandas as pd
from multiprocessing import cpu_count
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.optimizers import RMSprop ,Adam
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Subtract,Multiply
from keras.layers import Activation, Dense, Dropout, Input, Embedding,concatenate
from keras.layers.embeddings import Embedding

In [ ]:
from gensim.models import Word2Vec
from gensim.models.word2vec import Vocab

In [ ]:
#!pip install deepwalk

In [ ]:
train_data = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/train/train.csv')
train_feat = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/train/user_features.csv')

In [ ]:
test_data = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/test.csv')

In [ ]:
#test_data['node1_id']
list_test_nodes = list(np.unique(list(test_data['node1_id'].unique()) + list(test_data['node2_id'].unique())))

Subsetting the Train Data

In [ ]:
train_data_ones = train_data[train_data['is_chat']==1]

In [ ]:
train_data_zeros = train_data[train_data['is_chat']==0]

In [ ]:
train_data_sub_ones = train_data_ones[(train_data_ones['node1_id'].isin(list_test_nodes)) | 
                            (train_data_ones['node2_id'].isin(list_test_nodes))]

In [ ]:
train_data_sub_ones.index = range(0,len(train_data_sub_ones))

In [ ]:
train_data_sub_zeros = train_data_zeros[(train_data_zeros['node1_id'].isin(list_test_nodes)) | 
                            (train_data_zeros['node2_id'].isin(list_test_nodes))]

In [ ]:
train_data_sub_zeros.index = range(0,len(train_data_sub_zeros))

In [ ]:
train_data_sub_ones = train_data_sub_ones[0:72244]

In [ ]:
train_data_sub_zeros = train_data_sub_zeros[0:72244]

In [ ]:
train_data = pd.concat([train_data_sub_ones,train_data_sub_zeros],axis=0)

In [ ]:
train_data.shape

In [ ]:
y_train = train_data['is_chat']

In [ ]:
y_cat = to_categorical(y_train)

Generate Random Walk through Graph

In [ ]:
x_ones = train_data[train_data['is_chat']==1]
x_ones.index = range(0,len(x_ones))

In [ ]:
a = list(np.unique(list(train_data['node1_id'].unique()) + list(train_data['node2_id'].unique())))

In [ ]:
G = nx.Graph()
G.add_nodes_from(a)

In [ ]:
x_ls1=[]
for i in range(0,len(x_ones)) :
    x_ls1.append((x_ones['node1_id'].iloc[i],x_ones['node2_id'].iloc[i]))

In [ ]:
# add edges from txt 
G.add_edges_from(x_ls1)

In [ ]:
list(G.neighbors(7791327))

In [ ]:
# val = list(dict_nbr.values())
# z=[]
# for i in tqdm(val):
#     z.append(len(i))

In [ ]:
#Mapping for word2id
node2idx = {w: i  for i, w in enumerate(a)}

In [ ]:
len(node2idx)
#len(a)

In [ ]:
dict_nbr = {}
for i in tqdm(range(0,len(a))):
    node_id = a[i]
    #node_id_up = node2idx[a[i]]
    ls_nbr = 2*[node_id]+list(G.neighbors(node_id))
    
    if len(ls_nbr)<7:
        len_rem = 7 - len(ls_nbr)
        ls_nbr1 = len_rem*[node_id] + ls_nbr
    
    elif len(ls_nbr)>=7:
        ls_nbr1 = ls_nbr[0:7]
    
    dict_nbr[node_id]=ls_nbr1
    ls_nbr1=[]

Embedding Matrix

In [ ]:
#Embedding matrix creation
nb_nodes = len(node2idx)
embedding_matrix = np.zeros((nb_nodes, 13))

for node, i in tqdm(node2idx.items()):
    #print i
    if i >= nb_nodes:
        continue
    
    embedding_vector = list(train_feat[train_feat['node_id']==node].values[0][1:])
    embedding_matrix[i] = embedding_vector
        
print('Null node embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [ ]:
dict_nbr_upd={}
temp=[]
for node, ls in tqdm(dict_nbr.items()):
    #print i
    node_id = node
    for nb_nodes in ls:
        nb1 = node2idx[nb_nodes]
        temp.append(nb1)
    dict_nbr_upd[node_id] = temp
    temp =[]

In [ ]:
#dict_nbr_upd

In [ ]:
x_feat1=[]
x_feat2=[]


for i in tqdm(range(0,len(train_data))):
    
    node1 = train_data['node1_id'].iloc[i]
    node2 = train_data['node2_id'].iloc[i]
    
    x_feat1.append(dict_nbr_upd[node1])
    x_feat2.append(dict_nbr_upd[node2])
    
    

In [ ]:
embedding_matrix.shape

In [ ]:
max_len = 7

In [ ]:
conv_layers = [[128, 6, 2],
               [128, 5, 2],
               [128, 4, 2],
               [128, 3, 2]]

In [ ]:
def model_cnn_1D():
    
    #Input
    left_input =  Input(shape=(max_len,))
    right_input = Input(shape=(max_len,))
    
    #Embedding Matrix
    layer =  Embedding(136526, 13, weights=[embedding_matrix], input_length=max_len,trainable=True)
                                         
    
    # Embedded version of the inputs
    encoded_left =  layer(left_input)
    encoded_right = layer(right_input)
    
    #Convolution with Filter size 2
    #x3 = Conv1D(128, 2, activation='relu')(encoded_left)
    #x3 = MaxPooling1D(pool_size = 2)(x3)
    #x3 = Flatten()(x3)
    
    #Embedding for left
    for filter_num, filter_size, pooling_size in conv_layers:
        x1 = Conv1D(filter_num, filter_size)(encoded_left)
        x1 = MaxPooling1D(pool_size=pooling_size)(x1) 
        x1 = Activation('relu')(x1)
    x1 = Flatten()(x1)  

    
    #Embedding for right
    for filter_num, filter_size, pooling_size in conv_layers:
        x2 = Conv1D(filter_num, filter_size)(encoded_right)
        x2 = Activation('relu')(x2)
        x2 = MaxPooling1D(pool_size=pooling_size)(x2)  
    x2 = Flatten()(x2)  
    
    #Subtraction
    subtracted = Subtract()([x1, x2])
    
    #multiply
    multiply =  Multiply()([x1, x2])
    
    #Concatenating
    merged = concatenate([x1,x2,subtracted,multiply],axis=1)
    
    #Dense Layer
    layer_dense = Dense(128, activation='relu')(merged)
    layer_dense = Dropout(0.2)(layer_dense)
    
    #Output Layer
    loss_total = Dense(2,activation='softmax')(layer_dense)

    model = Model(inputs=[left_input,right_input],outputs=loss_total)
    return model

In [ ]:
model = model_cnn_1D()
model.summary()

In [ ]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = Adam(lr=0.001), metrics = [auc])

In [ ]:
#Deep Learning Model
history = model.fit([x_feat1,
                     x_feat2] ,
                     y_cat, batch_size = 1024, epochs = 10, validation_split=0.1,
                     class_weight = {0:2 ,1:40}, 
                     verbose = 1)


In [ ]:
#val[1116514]